# Comparing performance and accuracy of EM, IP and mix-SQP algorithms

In this example, we compare the runtime and accuracy of the EM algorithm, the mix-SQP algorithm, and the interior-point method implemented by the MOSEK commercial solver (and called via the `KWDual` function in the R package `REBayes`).

## Analysis setup

*Before attempting to run this Julia code, make sure your computer is properly set up to run this code by following the setup instructions in the README of the [git repository](https://github.com/stephenslab/mixsqp-paper).*

We begin by loading the Distributions, LowRankApprox and RCall Julia packages, as well as some function definitions used in the code chunks below.

In [1]:
using Distributions
using LowRankApprox
using RCall
using Random
using Printf
using LinearAlgebra
using SparseArrays
include("datasim.jl");
include("likelihood.jl");
include("mixEM.jl");
include("mixsqp_julia_v1.jl");
include("REBayes.jl");

Next, initialize the sequence of pseudorandom numbers.

In [2]:
Random.seed!(2019);

## Generate a small data set

Let's begin with a smaller example with 50,000 samples.

In [3]:
z = normtmixdatasim(round(Int,5e4));

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [4]:
sd = autoselectmixsd(z,nv = 20);
L  = normlikmatrix(z,sd = sd);
size(L)

(50000, 20)

## Fit mixture model 

First we run each of the optimization algorithms once to precompile the relevant functions.

In [5]:
outem  = mixEM(L,maxiter = 100);
outip  = REBayes(L);
outsqp = mixsqp(L,verbose = false);

┌ Warning: RCall.jl: Loading required package: Matrix
└ @ RCall /Users/yosikim/.julia/packages/RCall/RPlFw/src/io.jl:110


Next, let's fit the model using the three algorithms. 

In [6]:
@time xem, tem = mixEM(L,tol = 1e-4,maxiter = 1000);
@time xip, tip = REBayes(L);
@time outsqp   = mixsqp(L,verbose = false);

  5.592359 seconds (33.17 k allocations: 9.373 GiB, 21.34% gc time)
  1.468151 seconds (3.52 k allocations: 194.456 KiB)
  0.247536 seconds (119.37 k allocations: 187.969 MiB, 11.78% gc time)


The mix-SQP algorithm algorithm is much faster than the other two methods, with the EM being the slowest. 

Further, the quality of the IP and SQP solutions is very similar, whereas the EM solution is much worse: 

In [7]:
fem  = mixobjective(L,xem);
fip  = mixobjective(L,xip);
fsqp = mixobjective(L,outsqp["x"]);
fbest = minimum([fem fip fsqp]);
@printf "Difference between EM and best solutions:  %0.2e\n" fem - fbest
@printf "Difference between IP and best solutions:  %0.2e\n" fip - fbest
@printf "Difference between SQP and best solutions: %0.2e\n" fsqp - fbest

Difference between EM and best solutions:  1.24e+01
Difference between IP and best solutions:  1.21e-03
Difference between SQP and best solutions: 0.00e+00


## Comparison using a larger data set

Next, let's see what happens when we apply these three algorithms to a larger data set.

In [8]:
Random.seed!(2020);
z = normtmixdatasim(round(Int,1e5));

As before, we compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals. This time, we use a finer grid of $k = 100$ normal densities.

In [9]:
sd = autoselectmixsd(z,nv = 100);
L  = normlikmatrix(z,sd = sd);
size(L)

(100000, 100)

Now we fit the model using the three approaches. 

In [10]:
@time xem, tem = mixEM(L,tol = 1e-4,maxiter = 1000);
@time xip, tip = REBayes(L);
@time outsqp   = mixsqp(L,verbose = false);

 11.673978 seconds (2.48 k allocations: 11.327 GiB, 43.70% gc time)
 15.799090 seconds (265 allocations: 13.109 KiB)
  0.885132 seconds (680.13 k allocations: 469.331 MiB, 23.21% gc time)


In this example, the mix-SQP algorithm reaches a solution much faster than the both EM and IP approaches. 

As before, the quality of the IP and SQP solutions is similar, whereas the EM solution is much worse.

In [11]:
fem  = mixobjective(L,xem);
fip  = mixobjective(L,xip);
fsqp = mixobjective(L,outsqp["x"]);
fbest = minimum([fem fip fsqp]);
@printf "Difference between EM and best solutions:  %0.2e\n" fem - fbest
@printf "Difference between IP and best solutions:  %0.2e\n" fip - fbest
@printf "Difference between SQP and best solutions: %0.2e\n" fsqp - fbest

Difference between EM and best solutions:  1.15e+02
Difference between IP and best solutions:  4.47e-03
Difference between SQP and best solutions: 0.00e+00


## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, R and the packages used. 

In [14]:
using Pkg
Pkg.status()
versioninfo()

    Status `~/.julia/environments/v1.0/Project.toml`
  [7d9fca2a] Arpack v0.3.0
  [336ed68f] CSV v0.4.3
  [159f3aea] Cairo v0.5.6
  [a81c6b42] Compose v0.7.1
  [f65535da] Convex v0.11.3
  [a93c6f00] DataFrames v0.14.1
  [31c24e10] Distributions v0.16.4
  [5789e2e9] FileIO v1.0.6
  [186bb1d3] Fontconfig v0.2.0
  [f6369f11] ForwardDiff v0.10.3
  [c91e804a] Gadfly v1.0.0
  [2e9cd046] Gurobi v0.5.5
  [7073ff75] IJulia v1.14.1
  [916415d5] Images v0.18.0
  [b6b21f68] Ipopt v0.4.3
  [4076af6c] JuMP v0.18.4
  [67920dd8] KNITRO v0.5.2
  [093fc24a] LightGraphs v1.2.0
  [898213cb] LowRankApprox v0.2.1
  [e1d29d7a] Missings v0.4.0
  [6405355b] Mosek v0.9.11
  [9b87118b] PackageCompiler v0.5.1
  [2dcacdae] ParallelDataTransfer v0.5.0
  [91a5bcdd] Plots v0.21.0
  [d330b81b] PyPlot v2.6.3
  [dca85d43] QuartzImageIO v0.6.0
  [6f49c342] RCall v0.13.0
  [ce6b1742] RDatasets v0.6.1
  [f2b01f46] Roots v0.7.3
  [276daf66] SpecialFunctions v0.7.2
  [2913bbd2] StatsBase v0.26.0
  [4c63d2b9] StatsFuns v0.7.1

Since we called the `KWDual` function in R, it is also useful to record information about R.

In [13]:
R"sessionInfo()"

RObject{VecSxp}
R version 3.5.3 (2019-03-11)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.3

Matrix products: default
BLAS: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] REBayes_1.8   Matrix_1.2-17

loaded via a namespace (and not attached):
[1] compiler_3.5.3  Rmosek_8.0.69   grid_3.5.3      lattice_0.20-38
